### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

### Collecting Fashion House Tweets

In [2]:
#Reading in application keys 

with open('../data/twitter_keys.txt') as api:
    keys = api.read().split('\n') 

In [3]:
#Aplication keys

import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        keys[0],
    'consumer_secret':     keys[1],
    'access_token_key':    keys[2],
    'access_token_secret': keys[3]
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode           =   'extended'
)


In [4]:
statuses = api.GetUserTimeline(screen_name="gucci", count=200, trim_user=False)

In [7]:
tweet = statuses[0]
tweet.full_text

'Performing onstage with his band @30SECONDSTOMARS for ‘The Monolith Tour’ in Rome, @JaredLeto wore a #Gucci suit with oversized black velvet contrasting lapel and pinstripe sequin embroidery details with metallic leather high top sneakers. #AlessandroMichele #MonolithTour https://t.co/9Q5JI81CA8'

In [8]:
[status.full_text for status in statuses]

['Performing onstage with his band @30SECONDSTOMARS for ‘The Monolith Tour’ in Rome, @JaredLeto wore a #Gucci suit with oversized black velvet contrasting lapel and pinstripe sequin embroidery details with metallic leather high top sneakers. #AlessandroMichele #MonolithTour https://t.co/9Q5JI81CA8',
 'Spotted in @LOfficielES, @FinnSkata in a velvet formal jacket and retro shirt from #GucciSS18 by #AlessandroMichele. #GucciEditorials \nPhotography by: Graham Dunn\nStyling by: Danyl Geneciran https://t.co/RBQ56T1J8P',
 '.@Harry_Styles wore a #Gucci custom lurex jacquard suit with oversized peak lapel jacket and a silk self-tie blouse while performing onstage at Royal Arena in Copenhagen, Denmark during the second leg of ‘Harry Styles: Live on Tour’. #AlessandroMichele https://t.co/YLwLhLi3Uh',
 'Help @CHIMEFORCHANGE, @UNICEF and @beyonce empower women, girls and families in Burundi through access to clean, safe water, visit: https://t.co/4jKCASIntX. #beygood4burundi #wateris',
 'Gucci’s 

In [9]:
#Twitter API Class

class TweetMiner():
    
    result_limit = 20
    api          = None
    tweets       = []
    
    def __init__(self, keys_dict, api, result_limit = 20):
        
        self.keys_dict     = keys_dict
        self.api           = api
        self.results_limit = result_limit
        
    def mine_user_tweets(self, user='gucci', mine_retweets = False, max_pages = None, last_tweet_id = False):
        
        #time out feedback
        print("Mining tweets for:", user)
        
        tweets        = []
        
        page           = 1
        
        while page <= max_pages:
            
            #time out feedback
            print("Mining page:", page)
            
            if last_tweet_id:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit,
                    max_id      = last_tweet_id - 1
                )
            else:
                statuses = self.api.GetUserTimeline(
                    screen_name = user,
                    count       = self.result_limit
                )

            for item in statuses:
                
                last_tweet_id = item.id
                
                tweets.append({
                    'tweet_id': item.id,
                    'handle':   item.user.name,
                    'text':     item.full_text,
                    'created':  item.created_at
                })
                
            page += 1
        
        return tweets
            
        
miner = TweetMiner(twitter_keys, api, result_limit=50)

### Fashion Labels

In [10]:
wikipedia_labels = ['akris_official', 'McQueen', 'alfreddunhill', 'AlphaIndustries', 'andres_sarda', 'Anne_Fontaine',\
                   'aquascutum', 'giorgioarmani', 'emporioarmani', 'ArmaniExchange', 'AspinalofLondon', 'AspreyLondon',\
                   'AleDellAcqua', 'Dior', 'dkny', 'dolcegabbana', '_BadgleyMischka', 'BadgleyMischka', 'BALENCIAGA',\
                   'Bally', 'Balmain', 'BensonAndClegg', 'berluti', 'Beulahlondon', 'ManoloBlahnik', 'HUGOBOSS',\
                    'BottegaVeneta', 'ThomBrowneNY', 'Bulgariofficial', 'Burberry', '']

In [15]:
fashion_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Fendi', 'MaisonValentino', 'Roberto_Cavalli',\
                  'LouboutinWorld', 'EtroOfficial', 'Prada', 'dolcegabbana', 'Dior', 'LouisVuitton', 'CHANEL'\
                 'Missoni', 'OscardelaRenta', 'VeraWangGang', 'HerveLeger', 'MaisonValentino', 'Balmain',\
                 'Versace', 'giorgioarmani', 'DerekLamNYC', 'AlbertaFerretti', 'YSL', 'COMMEGARCONS', 'BALENCIAGA',\
                 'Burberry', 'TOMFORD', 'ALEXVAUTHIER', 'MIUMIUofficial', 'CalvinKlein', 'givenchy', 'TommyHilfiger',\
                 'jimmychoo', 'OffWht', 'chloefashion','dkny', 'marcjacobs', 'MichaelKors', 'Goyard', 'HUGOBOSS',\
                 'kenzo', 'HELMUTLANG', 'acnestudios', 'Bulgariofficial', 'akris_official', '']

prime_labels = ['gucci', 'McQueen', 'StellaMcCartney', 'Roberto_Cavalli', 'acnestudios', 'jimmychoo', 'Roberto_Cavalli',\
               'TOMFORD', 'Prada', 'akris_official']

print('fashion_labels:', len(fashion_labels))
print('prime_labels:', len(prime_labels))
print('wikipedia_labels', len(wikipedia_labels))

fashion_labels: 46
prime_labels: 10
wikipedia_labels 31


In [18]:
#test will just dump directly into database
#Instantiating class and passing list of twitter handles

tweet_data = []

for label in prime_labels:
    try:
        tweet_data = miner.mine_user_tweets(user=label, max_pages=50, last_tweet_id=True)
        df = pd.DataFrame(tweet_data)
        df.to_csv('../data/%s.csv' % label)
    except:
        print('could not mine label:', label)


Mining tweets for: gucci
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining page: 11
Mining page: 12
Mining page: 13
Mining page: 14
Mining page: 15
Mining page: 16
Mining page: 17
Mining page: 18
Mining page: 19
Mining page: 20
Mining page: 21
Mining page: 22
Mining page: 23
Mining page: 24
Mining page: 25
Mining page: 26
Mining page: 27
Mining page: 28
Mining page: 29
Mining page: 30
Mining page: 31
Mining page: 32
Mining page: 33
Mining page: 34
Mining page: 35
Mining page: 36
Mining page: 37
Mining page: 38
Mining page: 39
Mining page: 40
Mining page: 41
Mining page: 42
Mining page: 43
Mining page: 44
Mining page: 45
Mining page: 46
Mining page: 47
Mining page: 48
Mining page: 49
Mining page: 50
Mining tweets for: McQueen
Mining page: 1
Mining page: 2
Mining page: 3
Mining page: 4
Mining page: 5
Mining page: 6
Mining page: 7
Mining page: 8
Mining page: 9
Mining page: 10
Mining

In [16]:
#test will just dump directly into database
#Instantiating class and passing list of twitter handles

tweet_data = []

for label in prime_labels:
    tweet_data.extend(miner.mine_user_tweets(user=label, max_pages=250, last_tweet_id=True))

fashion = pd.DataFrame(tweet_data)

Mining tweets for: gucci
Mining page: 1


TwitterError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [13]:
fashion.shape

NameError: name 'fashion' is not defined

In [15]:
fashion.groupby(fashion.handle).size()

handle
Acne Studios         1000
Akris                 998
Alexander McQueen    1000
Jimmy Choo           1000
PRADA                1000
Roberto Cavalli      2000
Stella McCartney     1000
TOM FORD             1000
gucci                1000
dtype: int64

In [16]:
fashion.to_pickle('../data/fashion.pkl')

In [17]:
fashion.head()

,created,handle,text,tweet_id
0,Wed Mar 21 22:00:19 +0000 2018,gucci,A fresh and green scent Gucci Bloom Acqua di F...,976579281898885120
1,Wed Mar 21 20:00:19 +0000 2018,gucci,New scent Gucci Bloom Acqua di Fiori blends a ...,976549082427740160
2,Wed Mar 21 17:59:41 +0000 2018,gucci,"Presenting a new chapter of #InBloom, Gucci Bl...",976518722830716931
3,Tue Mar 20 19:00:35 +0000 2018,gucci,"“It was my way of surviving, to charm people o...",976171664982659073
4,Tue Mar 20 16:00:39 +0000 2018,gucci,".@MichaelClarkCo reveals, ""Japan is the most a...",976126380206383104


In [27]:
pd.read_csv('../data/akris_official.csv', index_col=[0])

,created,handle,text,tweet_id
0,Wed Mar 07 16:05:45 +0000 2018,Akris,Vienna 1900-1918. It was a groundbreaking time...,971416620429127680
1,Mon Mar 05 13:29:49 +0000 2018,Akris,"A closer look into our #AkrisFW18 collection, ...",970652605578870784
2,Sun Mar 04 17:18:36 +0000 2018,Akris,One of our powerful looks from the #AkrisFW18 ...,970347792932761600
3,Sat Mar 03 14:29:43 +0000 2018,Akris,Bonjour Paris! \nArt and architecture play an ...,969942904851566592
4,Mon Dec 04 14:40:51 +0000 2017,Akris,Albert Kriemler dedicated his #AkrisCR18 colle...,937693184464949248
5,Thu Nov 23 16:47:03 +0000 2017,Akris,"Inspired by the Cannes lifestyle, our new Akri...",933738678492549121
6,Thu Oct 05 07:04:51 +0000 2017,Akris,See the #AkrisSS18 show &amp; all Alexander Gi...,915835158326071296
7,Thu Oct 05 04:46:56 +0000 2017,Akris,Wood works: Akris carves out a collection insp...,915800448996708353
8,Tue Oct 03 13:40:44 +0000 2017,Akris,See more from our #AkrisSS18 collection now on...,915210006777253890
9,Sun Oct 01 18:11:24 +0000 2017,Akris,Kicking off the #AkrisSS18 show with an Alexan...,914553350435336193
